<h1 align=center><font size = 5>Machine Learning models for aiding the decision-making process in emergency departments</font></h1>

<h1>Tabla comparativa de algoritmos<h1>
<h3>Descripción</h3>
<p>
En este script se desarrollará una serie de predicciones basadas en datos del hospital San Juan de Dios Curicó, correspondientes al año 2018 representados por registros de urgencias. El objetivo es predecir, mediante un conjunto de algoritmos, la necesidad de hospitalización de un paciente de urgencias,tomando como input, datos proporcionados por el paciente en la etapa de registro, sus signos vitales registrados en la etapa de triage y el diagnóstico ofrecido por el médico tratante.
Se correrán algoritmos de prodicción tales como árboles y bosques de desición, regresión logística, support vector machines y redes neuronales. Para finalmente evaluar el rendimiento de cada algoritmo en términos de la predicción, mediante indicadores tales como Acuraccy, F1-Score, Curva ROC, Índice de Jaccard y logloss
</p>

<h1 id="Descripción de datos">Descripción de datos</h1>
<p>
Los datos utilizados fueros proporcionados por el Hospital San Juan de Dios, Curicó, Chile y corresponden a 4.971 registros de pacientes que asistieron a urgencias durante el periodo comprendido entre el 1 de enero de 2018 y agosto de 2019, los datos fueron limpiados y transformados en un script desarrollado previamente
<ul>
    <li>Datos: <a href="https://drive.google.com/open?id=1Bp7_MnK6cGwgBuwIq1a8S4DS_0wVDiAD" target="_blank">https://drive.google.com/open?id=1Bp7_MnK6cGwgBuwIq1a8S4DS_0wVDiAD</a></li>
    <li>Tipo de datos: csv</li>
   </ul>
<p>
Las variables presentes en los datos se describen a continuación:
<ul>    
   
   <li><b>PAC_EDAD</b>: corresponde a la edad del paciente en enteros</li>
   <li><b>MOTIVO_CONSULTA</b>: corresponde a la razón por la que el paciente acude al servicio de urgencias string</li>
   <li><b>MEDIO</b>: corresponde al medio de llegada, mediante el que el paciente acude al servicio de urgencias</li>
   <li><b>SEXO</b>: corresponde al sexo del paciente</li>
   <li><b>CAT</b>: corresponde a la categoría de gravedad asignada al paciente en el proceso de Triage</li>
   <li><b>PRESION_SIST</b>: corresponde la presión sistólica del paciente </li>
   <li><b>PRESION_DIAST</b>: corresponde la presióndiastólica del paciente</li>
   <li><b>SATO2</b>: Dato numérico que representa la saturometria del paciente (Nivel de oxigeno en la sangre)</li>
   <li><b>TEMPERATURA</b>: corresponde a la temperatura corporal del paciente en el momento de la categorización</li>
   <li><b>GLASGOW</b>: corresponde a al nivel registrado por el paciente en la escala Glasgow</li>
   <li><b>DM</b>: corresponde si el paciente presenta o no Diabetes Mellitus</li>
   <li><b>EVA</b>: corresponde si se aplica al paciente una evaluación de vias aéreas</li>
   <li><b>HGT</b>: corresponde a la medida de azucar en la sangre del paciente</li>
   <li><b>LCFA</b>: corresponde a si el paciente presenta obstrucción crónica de vías aéreas</li>
   <li><b>FR</b>: corresponde a la frecuencia respiratoria del paciente</li>
   <li><b>HTA</b>: corresponde a si el paciente posee Hipertención Arterial</li>
   <li><b>HORA_INSC</b>: corresponde a la hora en la que el paciente fue categorizado</li>
   <li><b>MIN_INSC</b>: corresponde al minuto en que el paciente fue categorizado</li>
   <li><b>TIEMPO_ESPERA_CAT</b>: corresponde al tiempo que espera el paciente luego de ser registrado, para ser categorizado</li>
      
</ul>
</p>

Cargar paquetes necesarios

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import sklearn as sk  
#import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
#from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
#Métodos de tuneo de parámetros
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Lectura de datos

In [ ]:
archivo= 'df_definitiva_B.csv'
df_hosp= pd.read_csv(archivo,encoding='latin-1',sep=",")

In [ ]:
archivo='df_cat_soloTexto.csv'
df_cat= pd.read_csv(archivo,encoding='latin-1',sep=",")

In [ ]:
df_hosp =df_hosp[['ID_PACIENTE','DESTINO']]

In [ ]:
df_cat.columns
df_cat.drop(columns =['DESC_EVENTO','CAT'], inplace = True)

Unión de bases de datos

In [ ]:
df_hosp =  df_hosp.join(df_cat, lsuffix='_caller', rsuffix='_other')

In [ ]:
# Limpiando de faltantes las columnas
df_hosp.dropna(subset=["ID_PACIENTE_caller"], axis=0, inplace = True)
df_hosp.dropna(subset=["ID_PACIENTE_other"], axis=0, inplace = True)

In [ ]:
df_hosp.drop(columns =['Unnamed: 0'], inplace = True)
df_cat.drop(columns =['Unnamed: 0'], inplace = True)

In [ ]:
df_hosp.shape

In [ ]:
# Limpiando de faltantes las columnas
df_hosp.dropna(subset=["DESTINO"], axis=0, inplace = True)

In [ ]:
#Quitando ID_PACIENTE
df_hosp.drop(columns =["ID_PACIENTE_caller"], inplace = True)
df_hosp.drop(columns =["ID_PACIENTE_other"], inplace = True)

In [ ]:
y =df_hosp['DESTINO']

In [ ]:
#Quitando 'DESTINO'
#df_hosp.drop(columns =['DESTINO'], inplace = True)
#df_hosp.drop(columns =['CAT'], inplace = True)
df_hosp.drop(columns =['DESTINO'], inplace = True)

In [ ]:
missing_data = df_hosp.isnull()

    
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

Seleccionar las variables relevantes para la predicción y asignarselas a la matriz X, de variables independientes

In [ ]:
X=df_hosp
X.shape

In [ ]:
y.shape

 <h1 id="Normalización de datos">Normalización de datos</h1>
<p>
Para aplicar el paquete de arboles de desición, los datos deben estar en una escala similar, es por ello que optamos por normalizarlos para que los valores estén en un rango entre -2 y 2. Esta medida no altera los resultados aunque si faborece a la eficiencia de los algoritmos
</p>

In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

 <h1 id="Configurando algoritmos">Configurando algoritmos</h1>
<p>
En esta sección se definen parámetros necesarios para la correcta aplicación de los algoritmos a implementar, además de seccionar el conjunto de datos en datos de prueba(30%)y de entrenamiento (70%). Los parámetros escogidos pueden ser modificados con el fin de obtener resultados diferentes
</p>

Carga de paquetes

In [ ]:
from sklearn.model_selection import train_test_split

Segmentación del conjunto de datos

In [ ]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

Resampling del conjunto de datos

In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
from collections import Counter
 
os_us = SMOTETomek(sampling_strategy='auto',random_state=None,smote=None,tomek=None,n_jobs=None,)
X_trainset_res_ST1, y_trainset_res_ST1 = os_us.fit_sample(X_trainset, y_trainset)
 
print ("Distribution before resampling {}".format(Counter(y_trainset)))
print ("Distribution labels after resampling {}".format(Counter(y_trainset_res_ST1)))

Árbol de desición

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT= DecisionTreeClassifier(criterion="entropy", max_depth = 4)
DT.fit(X_trainset,y_trainset)
yhat_1 = DT.predict(X_testset)
yhat_prob_1=DT.predict_proba(X_testset)
DT_Acc=round(metrics.accuracy_score(y_testset, yhat_1),4)
#DT_Jcc=round(jaccard_similarity_score(y_testset, yhat_1),4)
DT_lgl=round(log_loss(y_testset, yhat_prob_1),4)
DT_F1=f1_score(y_testset, yhat_1, average='weighted') 

In [ ]:
DT_GS = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0,
                       random_state=42, splitter='best')
DT_GS.fit(X_trainset,y_trainset)
DT_GS_ST=DT_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
DT_parameters = [{'criterion': ['entropy', 'gini'], 'max_depth': np.arange(3, 21)},{'min_samples_leaf': [5, 10, 20, 50, 100]}]
DT_GS = GridSearchCV(DecisionTreeClassifier(random_state=42), DT_parameters, verbose=1, cv=5, scoring='balanced_accuracy')
DT_GS.fit(X_trainset,y_trainset)
DT_GS_ST=DT_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
DT_GS.best_estimator_

In [ ]:
yhat_1_GS= DT_GS.predict(X_testset)
yhat_prob_1_GS=DT_GS.predict_proba(X_testset)
DT_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_1_GS),4)
#DT_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_1_GS),4)
DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS),4)
DT_F1_GS=round(f1_score(y_testset, yhat_1_GS, average='weighted'),4) 

In [ ]:
yhat_1_GS_ST= DT_GS_ST.predict(X_testset)
yhat_prob_1_GS=DT_GS.predict_proba(X_testset)
DT_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_1_GS_ST),4)
#DT_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_1_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
DT_F1_GS_ST=round(f1_score(y_testset, yhat_1_GS_ST, average='weighted'),4) 

In [ ]:
resultados_DT = {'índices de rendimiento':['Accuracy','F1-Score'],
             'Árboles de decisión':[DT_Acc,DT_F1],
             'Grid Search':[DT_Acc_GS,DT_F1_GS],
                'GS+SMOTE-Tomek':[DT_Acc_GS_ST,DT_F1_GS_ST]}
Tabla_resultados_DT=pd.DataFrame(resultados_DT)
print(Tabla_resultados_DT)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
X=df_hosp

In [ ]:
perm = PermutationImportance(DT, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names =X.columns.tolist())

In [ ]:
import shap
ex = shap.TreeExplainer(DT)
shap_values = ex.shap_values(X)
shap.summary_plot(shap_values, X)

Bosque de desición

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF= RandomForestClassifier(max_depth=2, random_state=0)
RF.fit(X_trainset,y_trainset)
yhat_2 = RF.predict(X_testset)
yhat_prob_2=RF.predict_proba(X_testset)
RF_Acc=round(metrics.accuracy_score(y_testset, yhat_2),4)
#RF_Jcc=round(jaccard_similarity_score(y_testset, yhat_2),4)
RF_lgl=round(log_loss(y_testset, yhat_prob_2),4)
RF_F1=f1_score(y_testset, yhat_2, average='weighted') 

In [ ]:
RF_GS =RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
RF_GS.fit(X_trainset,y_trainset)
DT_GS_ST=DT_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
RF_parameters ={ 'n_estimators': [100,200,300,400,500],'criterion': ['entropy', 'gini'], 'max_depth': np.arange(3, 21)},{'min_samples_leaf': [5, 10, 20, 50, 100]}
RF_GS = GridSearchCV(estimator=RF,param_grid=RF_parameters, cv= 3,scoring="accuracy")
RF_GS.fit(X_trainset,y_trainset)
RF_GS_ST=RF_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
RF_GS.best_estimator_

In [ ]:
yhat_2_GS= RF_GS.predict(X_testset)
yhat_prob_2_GS=RF_GS.predict_proba(X_testset)
RF_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_2_GS),4)
#RF_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_2_GS),4)
#RF_lgl_GS=round(log_loss(y_testset, yhat_prob_2_GS),4)
RF_F1_GS=round(f1_score(y_testset, yhat_2_GS, average='weighted'),4) 

In [ ]:
yhat_2_GS_ST= DT_GS_ST.predict(X_testset)
yhat_prob_2_GS=DT_GS.predict_proba(X_testset)
RF_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_2_GS_ST),4)
#RF_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_2_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
RF_F1_GS_ST=round(f1_score(y_testset, yhat_2_GS_ST, average='weighted'),4) 

In [ ]:
resultados_RF = {'índices de rendimiento':['Accuracy','F1-Score'],
             'Bosque de decisión':[RF_Acc,RF_F1],
             'Grid Search':[RF_Acc_GS,RF_F1_GS],
                'GS+SMOTE-Tomek':[RF_Acc_GS_ST,RF_F1_GS_ST]}
Tabla_resultados_RF=pd.DataFrame(resultados_RF)
print(Tabla_resultados_RF)

In [ ]:
import shap
ex = shap.TreeExplainer(RF)
shap_values = ex.shap_values(X)
shap.summary_plot(shap_values, X)

Regresión logística (multi-class)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='newton-cg',penalty='l2',multi_class='ovr').fit(X_trainset,y_trainset)
yhat_3= LR.predict(X_testset)
yhat_prob_3=LR.predict_proba(X_testset)
LR_Acc=round(metrics.accuracy_score(y_testset, yhat_3),4)
#LR_Jcc=round(jaccard_similarity_score(y_testset, yhat_3),4)
LR_lgl=round(log_loss(y_testset, yhat_prob_3),4)
LR_F1=f1_score(y_testset, yhat_3, average='weighted') 

In [ ]:
LR_GS = LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)
LR_GS.fit(X_trainset,y_trainset)
LR_GS_ST=LR_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
LR_parameters ={'C':(0.1,1,10), 'penalty':['l1','l2','elasticnet']}
LR_GS = GridSearchCV(estimator=LR,param_grid=LR_parameters, verbose=True, n_jobs=-1,cv=3)
LR_GS.fit(X_trainset,y_trainset)
LR_GS_ST=LR_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
LR_GS.best_estimator_

In [ ]:
yhat_3_GS= LR_GS.predict(X_testset)
yhat_prob_3_GS=LR_GS.predict_proba(X_testset)
LR_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_3_GS),4)
#LR_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_3_GS),4)
LR_lgl_GS=round(log_loss(y_testset, yhat_prob_3_GS),4)
LR_F1_GS=round(f1_score(y_testset, yhat_3_GS, average='weighted'),4) 

In [ ]:
yhat_3_GS= LR_GS.predict(X_testset)
yhat_prob_3_GS=LR_GS.predict_proba(X_testset)
LR_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_3_GS),4)
#LR_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_3_GS),4)
LR_lgl_GS=round(log_loss(y_testset, yhat_prob_3_GS),4)
LR_F1_GS=round(f1_score(y_testset, yhat_3_GS, average='weighted'),4) 

In [ ]:
yhat_3_GS_ST= LR_GS_ST.predict(X_testset)
#yhat_prob_2_GS=DT_GS.predict_proba(X_testset)
LR_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_3_GS_ST),4)
#LR_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_3_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
LR_F1_GS_ST=round(f1_score(y_testset, yhat_3_GS_ST, average='weighted'),4) 

In [ ]:
resultados_LR = {'índices de rendimiento':['Accuracy','F1-Score'],
             'Regresión Logística':[LR_Acc,LR_F1],
             'Grid Search':[LR_Acc_GS,LR_F1_GS],
                 'GS+SMOTE-Tomek':[LR_Acc_GS_ST,LR_F1_GS_ST]}
Tabla_resultados_LR=pd.DataFrame(resultados_LR)
print(Tabla_resultados_LR)

In [ ]:
# Kerner Explainer
explainer = shap.KernelExplainer(LR.predict_proba,X[:100])
shap_values = explainer.shap_values(X[:100])
shap.summary_plot(shap_values, X[:100])

Redes Neuronales

In [ ]:
from sklearn.neural_network import MLPClassifier
NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1).fit(X_trainset,y_trainset)
yhat_4 = NN.predict(X_testset)
yhat_prob_4=NN.predict_proba(X_testset)
NN_Acc=round(metrics.accuracy_score(y_testset, yhat_4),4)
#NN_Jcc=round(jaccard_similarity_score(y_testset, yhat_3),4)
NN_lgl=round(log_loss(y_testset, yhat_prob_4),4)
NN_F1=f1_score(y_testset, yhat_4, average='weighted') 

In [ ]:
NN_GS = MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 100, 50), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
NN_GS.fit(X_trainset,y_trainset)
NN_GS_ST=NN_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
NN_parameters = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],'alpha': [0.0001, 0.05],'learning_rate': ['constant','adaptive']}

NN_GS = GridSearchCV(estimator=NN,param_grid=NN_parameters, cv= 3, verbose=True,scoring="accuracy")
NN_GS.fit(X_trainset,y_trainset)
NN_GS_ST=NN_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
NN_GS.best_estimator_

In [ ]:
yhat_4_GS= NN_GS.predict(X_testset)
#yhat_prob_4_GS=NN_GS.predict_proba(X_testset)
NN_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_4_GS),4)
#NN_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_4_GS),4)
#NN_lgl_GS=round(log_loss(y_testset, yhat_prob_4_GS),4)
NN_F1_GS=round(f1_score(y_testset, yhat_4_GS, average='weighted'),4) 

In [ ]:
yhat_4_GS_ST= NN_GS_ST.predict(X_testset)
#yhat_prob_2_GS=DT_GS.predict_proba(X_testset)
NN_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_4_GS_ST),4)
#NN_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_4_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
NN_F1_GS_ST=round(f1_score(y_testset, yhat_4_GS_ST, average='weighted'),4) 

In [ ]:
resultados_NN = {'índices de rendimiento':['Accuracy','F1-Score'],
             'Red Neuronal':[NN_Acc,NN_F1],
             'Grid Search':[NN_Acc_GS,NN_F1_GS],
                'GS+SMOTE-Tomek':[NN_Acc_GS_ST,NN_F1_GS_ST]}
Tabla_resultados_NN=pd.DataFrame(resultados_NN)
print(Tabla_resultados_NN)

In [ ]:
data_DF=pd.DataFrame(X, columns=df_hosp.columns)
data_DF

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
perm = PermutationImportance(NN, random_state=1).fit(data_DF, y)
eli5.show_weights(perm, feature_names =data_DF.columns.tolist())

In [ ]:
# Kerner Explainer
import shap
explainer = shap.KernelExplainer(NN.predict_proba,data_DF[:100])
shap_values = explainer.shap_values(data_DF[:100])
shap.summary_plot(shap_values, data_DF[:100])

Support Vector Machine

In [ ]:
from sklearn import svm
SVM = svm.SVC(kernel='rbf',decision_function_shape='ovo', probability=True)
SVM.fit(X_trainset, y_trainset) 
yhat_5 = SVM.predict(X_testset)
yhat_prob_5=SVM.predict_proba(X_testset)
SVM_Acc=round(metrics.accuracy_score(y_testset, yhat_5),4)
#SVM_Jcc=round(jaccard_similarity_score(y_testset, yhat_5),4)
SVM_lgl=round(log_loss(y_testset, yhat_prob_5),4)
SVM_F1=f1_score(y_testset, yhat_5, average='weighted') 

In [ ]:
SVM_parameters ={'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
SVM_GS = GridSearchCV(estimator=SVM,param_grid=SVM_parameters, cv= 3, verbose=True, n_jobs=-1)
SVM_GS.fit(X_trainset,y_trainset)
SVM_GS_ST=SVM_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
SVM_GS.best_estimator_

In [ ]:
yhat_5_GS= SVM_GS.predict(X_testset)
yhat_prob_5_GS=SVM_GS.predict_proba(X_testset)
SVM_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_5_GS),4)
#SVM_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_5_GS),4)
SVM_lgl_GS=round(log_loss(y_testset, yhat_prob_5_GS),4)
SVM_F1_GS=round(f1_score(y_testset, yhat_5_GS, average='weighted'),4) 

In [ ]:
yhat_5_GS_ST= NN_GS_ST.predict(X_testset)
#yhat_prob_2_GS=DT_GS.predict_proba(X_testset)
SVM_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_5_GS_ST),4)
SVM_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_5_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
SVM_F1_GS_ST=round(f1_score(y_testset, yhat_5_GS_ST, average='weighted'),4) 

In [ ]:
resultados_SVM = {'índices de rendimiento':['Accuracy','Jaccard','F1-Score'],
             'Red Neuronal':[SVM_Acc,SVM_Jcc,SVM_F1],
             'Grid Search':[SVM_Acc_GS,SVM_Jcc_GS,SVM_F1_GS],
             'GS+SMOTE-Tomek':[SVM_Acc_GS_ST,SVM_Jcc_GS_ST,SVM_F1_GS_ST]}
Tabla_resultados_SVM=pd.DataFrame(resultados_SVM)
print(Tabla_resultados_SVM)

In [ ]:
# Kerner Explainer
explainer = shap.KernelExplainer(SVM.predict_proba,X[:100])
shap_values = explainer.shap_values(X[:100])
shap.summary_plot(shap_values, X[:100])

In [ ]:
resultados = {'Algoritmos de clasificación':['Árboles de decisión','AD+GS','Bosques de decisión','BD+GS','Regresión Logística','RL+GS','Red Neuronal','RN+GS'],
             'Accuracy':[DT_Acc,DT_Acc_GS,RF_Acc,RF_Acc_GS,LR_Acc,LR_Acc_GS,NN_Acc,NN_Acc_GS],
                     'F1-Score':[DT_F1,DT_F1_GS,RF_F1,RF_F1_GS,LR_F1,LR_F1_GS,NN_F1,NN_F1_GS]}
Tabla_resultados=pd.DataFrame(resultados)
print(Tabla_resultados)


 <h1 id="Matrices de confusión">Matrices de confusión</h1>
<p>
En esta sección se construyen matrices de confusión para evaluar la eficacia de cada algoritmo
</p>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de confusión',
                          cmap=plt.cm.Blues):
    """
    Esta función muestra y dibuja la matriz de confusión.
    La normalización se puede aplicar estableciendo el valor `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusión normalizada")
    else:
        print('Matriz de confusión sin normalización')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Etiqueta Real')
    plt.xlabel('Etiqueta Predicha')


Árbol de decisión

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_1, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión árbol de decisión')

Bosque de decisión

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_2_GS_ST, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión bosque de decisión + ST')

Regresión Logística

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_3_GS_ST, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión regresión logística +ST')

Red neuronal

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_4_GS_ST, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión red neuronal +ST')

Support Vector Machine

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_5, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión SVM')